# 선형 회귀(Linear Regression)
1. 선형 회귀
2. 자동 미분
3. 다중 선형 회귀
4. nn.Module로 구현하는 선형 회귀
5. 클래스로 파이토치 모델 구현하기
6. 미니 배치와 데이터 로드
7. 커스텀 데이터셋

---

## 선형회귀

### 선형 회귀의 구현

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# 현재 실습하고 있는 파이썬 코드를 재실행해도 다음에도 같은 결과가 나오도록 랜덤 시드(random seed)를 줍니다.
torch.manual_seed(1)

##### 1. 변수선언

In [3]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

##### 2. 가중치와 편향의 초기화

In [6]:
# 가중치 W를 0으로 초기화하고 학습을 통해 값이 변경되는 변수임을 명시함.
W = torch.zeros(1, requires_grad=True) 
b = torch.zeros(1, requires_grad=True)
# 가중치 W를 출력
print(W) 
print(b)

tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)


##### 3. 가설 설정

In [7]:
hypothesis = W * x_train + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


##### 4. 비용 함수 선언

In [9]:
cost = torch.mean((hypothesis - y_train) ** 2) 
print(cost)

tensor(18.6667, grad_fn=<MeanBackward0>)


##### 5. 경사 하강법 구현

In [10]:
optimizer = optim.SGD([W, b], lr = 0.01)

In [11]:
""" 항상 따라다니는 코드(epoch마다)
# gradient를 0으로 초기화
optimizer.zero_grad() 
# 비용 함수를 미분하여 gradient 계산
cost.backward() 
# W와 b를 업데이트
optimizer.step() 
"""

In [12]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])
# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 2000 # 원하는만큼 경사 하강법을 반복
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x_train * W + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch    0/2000 W: 0.187, b: 0.080 Cost: 18.666666
Epoch  100/2000 W: 1.746, b: 0.578 Cost: 0.048171
Epoch  200/2000 W: 1.800, b: 0.454 Cost: 0.029767
Epoch  300/2000 W: 1.843, b: 0.357 Cost: 0.018394
Epoch  400/2000 W: 1.876, b: 0.281 Cost: 0.011366
Epoch  500/2000 W: 1.903, b: 0.221 Cost: 0.007024
Epoch  600/2000 W: 1.924, b: 0.174 Cost: 0.004340
Epoch  700/2000 W: 1.940, b: 0.136 Cost: 0.002682
Epoch  800/2000 W: 1.953, b: 0.107 Cost: 0.001657
Epoch  900/2000 W: 1.963, b: 0.084 Cost: 0.001024
Epoch 1000/2000 W: 1.971, b: 0.066 Cost: 0.000633
Epoch 1100/2000 W: 1.977, b: 0.052 Cost: 0.000391
Epoch 1200/2000 W: 1.982, b: 0.041 Cost: 0.000242
Epoch 1300/2000 W: 1.986, b: 0.032 Cost: 0.000149
Epoch 1400/2000 W: 1.989, b: 0.025 Cost: 0.000092
Epoch 1500/2000 W: 1.991, b: 0.020 Cost: 0.000057
Epoch 1600/2000 W: 1.993, b: 0.016 Cost: 0.000035
Epoch 1700/2000 W: 1.995, b: 0.012 Cost: 0.000022
Epoch 1800/2000 W: 1.996, b: 0.010 Cost: 0.000013
Epoch 1900/2000 W: 1.997, b: 0.008 Cost: 0.000008

---

### 자동 미분 실습하기

##### 1. 자동 미분(Autograd) 실습하기

In [13]:
# 텐서에 대한 기울기를 저장하겠다는 의미
w = torch.tensor(2.0, requires_grad=True)

y = w**2
z = 2*y + 5

# 해당 수식을 w에 대하여 미분
z.backward()

print('수식을 w로 미분한 값 : {}'.format(w.grad))

수식을 w로 미분한 값 : 8.0


---

### 다중 선형 회귀

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [17]:
# 훈련 데이터
x1_train = torch.FloatTensor([[77], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[88], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [18]:
# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [4]:
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w3.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.294 w2: 0.297 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.674 w2: 0.676 w3: 0.676 b: 0.008 Cost: 1.563634
Epoch  200/1000 w1: 0.679 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.497603
Epoch  300/1000 w1: 0.684 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.435026
Epoch  400/1000 w1: 0.689 w2: 0.678 w3: 0.678 b: 0.008 Cost: 1.375730
Epoch  500/1000 w1: 0.694 w2: 0.678 w3: 0.678 b: 0.009 Cost: 1.319503
Epoch  600/1000 w1: 0.699 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.266215
Epoch  700/1000 w1: 0.704 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.215693
Epoch  800/1000 w1: 0.709 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.167821
Epoch  900/1000 w1: 0.713 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.122419
Epoch 1000/1000 w1: 0.718 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.079375


### 행렬 연산을 고려하여 파이토치로 구현하기

In [39]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros((5, 1), requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    # 편향 b는 브로드 캐스팅되어 각 샘플에 더해집니다.
    hypothesis = x_train.matmul(W) + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2550, 80.8370, 79.6496, 86.7367, 61.6576]) Cost: 9299.041992
Epoch    2/20 hypothesis: tensor([104.9094, 126.0961, 124.2437, 135.2988,  96.1786]) Cost: 2916.040283
Epoch    3/20 hypothesis: tensor([125.9912, 151.4358, 149.2109, 162.4876, 115.5064]) Cost: 915.195618
Epoch    4/20 hypothesis: tensor([137.7942, 165.6231, 163.1894, 177.7099, 126.3278]) Cost: 288.001404
Epoch    5/20 hypothesis: tensor([144.4023, 173.5665, 171.0157, 186.2326, 132.3867]) Cost: 91.397415
Epoch    6/20 hypothesis: tensor([148.1018, 178.0139, 175.3974, 191.0042, 135.7791]) Cost: 29.768749
Epoch    7/20 hypothesis: tensor([150.1729, 180.5040, 177.8505, 193.6756, 137.6786]) Cost: 10.449845
Epoch    8/20 hypothesis: tensor([151.3323, 181.8983, 179.2240, 195.1713, 138.7423]) Cost: 4.393543
Epoch    9/20 hypothesis: tensor([151.9812, 182.6791, 179.9928, 196.0086, 139.3380]) Cost: 2.494627
Epoch   10/20 hypo

---

### nn.Module로 구현하는 선형 회귀

##### 1. 단순 선형 회귀 구현하기

In [110]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

In [111]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [112]:
model = nn.Linear(1,1)
print(list(model.parameters()))

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]


In [113]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [115]:
nb_epochs = 2000
for epoch in range(1, nb_epochs+1):
    prediction = model(x_train)
    
    cost = F.mse_loss(prediction, y_train)
    
    optimizer.zero_grad
    cost.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:6f}'.format(
        epoch, nb_epochs, cost.item()))
        #print(list(model.parameters()))

Epoch  100/2000 Cost: 1.930341
Epoch  200/2000 Cost: 5.693942
Epoch  300/2000 Cost: 12.762090
Epoch  400/2000 Cost: 0.779174
Epoch  500/2000 Cost: 7.644921
Epoch  600/2000 Cost: 11.642735
Epoch  700/2000 Cost: 0.132139
Epoch  800/2000 Cost: 9.513975
Epoch  900/2000 Cost: 10.118929
Epoch 1000/2000 Cost: 0.033313
Epoch 1100/2000 Cost: 11.153074
Epoch 1200/2000 Cost: 8.312744
Epoch 1300/2000 Cost: 0.492536
Epoch 1400/2000 Cost: 12.427568
Epoch 1500/2000 Cost: 6.369245
Epoch 1600/2000 Cost: 1.480140
Epoch 1700/2000 Cost: 13.220701
Epoch 1800/2000 Cost: 4.461087
Epoch 1900/2000 Cost: 2.904804
Epoch 2000/2000 Cost: 13.472202


In [116]:
print(list(model.parameters()))
print(model(torch.FloatTensor([[20]])))

[Parameter containing:
tensor([[0.6347]], requires_grad=True), Parameter containing:
tensor([-0.5615], requires_grad=True)]
tensor([[12.1328]], grad_fn=<AddmmBackward>)


##### 2. 다중 선형 회귀 구현하기

In [117]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

In [118]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [124]:
model = torch.nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)
nb_epochs = 2000

for epoch in range(1, nb_epochs):
    cost = torch.nn.functional.mse_loss(y_train, model(x_train))
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch  100/2000 Cost: 1.166825
Epoch  200/2000 Cost: 1.142451
Epoch  300/2000 Cost: 1.119203
Epoch  400/2000 Cost: 1.097066
Epoch  500/2000 Cost: 1.075957
Epoch  600/2000 Cost: 1.055810
Epoch  700/2000 Cost: 1.036598
Epoch  800/2000 Cost: 1.018275
Epoch  900/2000 Cost: 1.000767
Epoch 1000/2000 Cost: 0.984056
Epoch 1100/2000 Cost: 0.968092
Epoch 1200/2000 Cost: 0.952831
Epoch 1300/2000 Cost: 0.938250
Epoch 1400/2000 Cost: 0.924303
Epoch 1500/2000 Cost: 0.910958
Epoch 1600/2000 Cost: 0.898189
Epoch 1700/2000 Cost: 0.885971
Epoch 1800/2000 Cost: 0.874267
Epoch 1900/2000 Cost: 0.863051


In [125]:
print(list(model.parameters()))

new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

[Parameter containing:
tensor([[0.6186, 0.4198, 0.9665]], requires_grad=True), Parameter containing:
tensor([0.1893], requires_grad=True)]
훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[151.4248]], grad_fn=<AddmmBackward>)


---

### 05. 클래스로 파이토치 모델 구현하기

##### 1. 모델을 클래스로 구현하기

In [126]:
class LinearRegressionModel(nn.Module): # torch.nn.Module을 상속받는 파이썬 클래스
    def __init__(self): #
        super().__init__()
        self.linear = nn.Linear(1, 1) # 단순 선형 회귀이므로 input_dim=1, output_dim=1.

    def forward(self, x):
        return self.linear(x)

In [127]:
model = LinearRegressionModel()

In [130]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
    def forward(self, x):
        return self.linear(x)

In [131]:
model = MultivariateLinearRegressionModel()

##### 2. 단순 선형 회귀 클래스로 구현하기

In [134]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
    def forward(self, x):
        return self.linear(x)
    
model = LinearRegressionModel()

optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

nb_epochs = 2000
for epoch in range(nb_epochs+1):
    h = model.forward(x_train)
    cost = nn.functional.mse_loss(h, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 13.103540
Epoch  100/2000 Cost: 0.002791
Epoch  200/2000 Cost: 0.001724
Epoch  300/2000 Cost: 0.001066
Epoch  400/2000 Cost: 0.000658
Epoch  500/2000 Cost: 0.000407
Epoch  600/2000 Cost: 0.000251
Epoch  700/2000 Cost: 0.000155
Epoch  800/2000 Cost: 0.000096
Epoch  900/2000 Cost: 0.000059
Epoch 1000/2000 Cost: 0.000037
Epoch 1100/2000 Cost: 0.000023
Epoch 1200/2000 Cost: 0.000014
Epoch 1300/2000 Cost: 0.000009
Epoch 1400/2000 Cost: 0.000005
Epoch 1500/2000 Cost: 0.000003
Epoch 1600/2000 Cost: 0.000002
Epoch 1700/2000 Cost: 0.000001
Epoch 1800/2000 Cost: 0.000001
Epoch 1900/2000 Cost: 0.000000
Epoch 2000/2000 Cost: 0.000000


### 3. 다중 선형 회귀 클래스로 구현하기

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

In [2]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [3]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 다중 선형 회귀이므로 input_dim=3, output_dim=1.

    def forward(self, x):
        return self.linear(x)

In [4]:
model = MultivariateLinearRegressionModel()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 
nb_epochs = 2000
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)
    # model(x_train)은 model.forward(x_train)와 동일함.

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 31667.597656
Epoch  100/2000 Cost: 0.225993
Epoch  200/2000 Cost: 0.223911
Epoch  300/2000 Cost: 0.221941
Epoch  400/2000 Cost: 0.220059
Epoch  500/2000 Cost: 0.218271
Epoch  600/2000 Cost: 0.216575
Epoch  700/2000 Cost: 0.214950
Epoch  800/2000 Cost: 0.213413
Epoch  900/2000 Cost: 0.211952
Epoch 1000/2000 Cost: 0.210559
Epoch 1100/2000 Cost: 0.209230
Epoch 1200/2000 Cost: 0.207967
Epoch 1300/2000 Cost: 0.206762
Epoch 1400/2000 Cost: 0.205618
Epoch 1500/2000 Cost: 0.204529
Epoch 1600/2000 Cost: 0.203481
Epoch 1700/2000 Cost: 0.202486
Epoch 1800/2000 Cost: 0.201539
Epoch 1900/2000 Cost: 0.200634
Epoch 2000/2000 Cost: 0.199770
